In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist
from neural_map import NeuralMap, _plot

In [ ]:
train_features = np.load('datasets/train_features.npy')
train_labels = np.load('datasets/train_labels.npy')
test_features = np.load('datasets/test_features.npy')
test_labels = np.load('datasets/test_labels.npy')

train_features.shape, train_labels.shape, test_features.shape, test_labels.shape

In [ ]:
scaler = MinMaxScaler()

s_train_features = scaler.fit_transform(train_features)
s_test_features = scaler.transform(test_features)

s_train_features.max(), s_train_features.min(), s_test_features.max(), s_test_features.min()

In [ ]:
dictionary = {}

with open('datasets/features_training_correlation_10.json') as json_file: 
    dictionary = json.load(json_file)
    dictionary['relative_positions'] = np.array(dictionary['relative_positions'])
    dictionary['weights'] = np.array(dictionary['weights'])
    
nm = NeuralMap(**dictionary)

In [ ]:
nm.plot_unified_distance_matrix(detailed=True, borders=True)

In [ ]:
nm.plot_analysis(data=s_train_features)

In [ ]:
nm.plot_analysis(data=s_test_features)

In [ ]:
nm.plot_labels(data=s_train_features, labels=train_labels)

In [ ]:
nm.plot_labels(data=s_test_features, labels=test_labels)

In [ ]:
# guarda la distancia euclidea (o la que uses) de cada feature de entrenamiento con su nodo ganador.

train_quantization_error = np.ones(s_train_features.shape[0]) * np.nan
for i in range(s_train_features.shape[0]):
    train_quantization_error[i] = nm.generate_activation_map(s_train_features[i]).min()

In [ ]:
# Luego con las imagenes de testing lo que hacemos es:
# 1) encontrar el nodo ganador
# 2) calcular la distancia entre la imagen(features) de testing contra las imagenes(features) del conjunto de entrenamiento que mapean a ese nodo.
# 3) mostramos una imagen y la otra al lado.
# Un ejemplo simple es, hacer esos pasos con una imagen de entrenamiento y recuerar la clase de la misma.

selected_feature_index = 412

if not ('mapped_train_features_indices' in vars() or 'mapped_train_features_indices' in globals()):
    mapped_train_features_indices = nm.map_attachments(s_train_features, np.array(range(s_train_features.shape[0])))
selected_feature = s_test_features[selected_feature_index]
bmu = nm.get_best_matching_unit(selected_feature)
train_features_indices = mapped_train_features_indices[bmu]
distances = cdist(selected_feature[None], s_train_features[train_features_indices], nm.metric)[0]

print("\n\n\nSelecetd feature idnex:\n\n", selected_feature_index)
print("\n\n\nSelecetd feature:\n\n", selected_feature)
print("\n\n\nSelected feature class:\n\n", test_labels[selected_feature_index])
print("\n\n\nBest matching unit:\n\n", bmu)
print("\n\n\nIndices of training features that share the same node:\n\n", mapped_train_features_indices[bmu])
print("\n\n\nClasses of the training features that shares same node:\n\n", train_labels[train_features_indices])
print("\n\n\nDistances from selected features to training features:\n\n", distances)